## Import Libraries

In [3]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.append('..')
from utils.plot import *

## Load the dataset

In [6]:
# Load testing data
data = pd.read_csv('https://raw.githubusercontent.com/ktxdev/mind-matters/refs/heads/master/data/raw/test.csv')

## Initial Exploration
### Shape and Structure

In [7]:
print(data.shape)
data.head()

(93800, 19)


,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
1,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
2,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
3,140703,Nalini,Female,23.0,Rajkot,Student,NaN,5.0,NaN,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
4,140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No


### Dropping Unwanted Features
- **id:** This is just a unique identifier for each person in the dataset, which does not contribute to predicting mental health outcomes.
- **Name:** This feature is not relevant to predicting depression and could introduce noise i.e it does not provide meaningful information for the model

In [8]:
data.drop(columns=['id', 'Name'], inplace=True)

### Rename columns

In [10]:
data.rename(columns={'Have you ever had suicidal thoughts ?': 'Had Suicidal Thoughts?'}, inplace=True)

## Data Types

In [11]:
data.dtypes

Gender                               object
Age                                 float64
City                                 object
Working Professional or Student      object
Profession                           object
Academic Pressure                   float64
Work Pressure                       float64
CGPA                                float64
Study Satisfaction                  float64
Job Satisfaction                    float64
Sleep Duration                       object
Dietary Habits                       object
Degree                               object
Had Suicidal Thoughts?               object
Work/Study Hours                    float64
Financial Stress                    float64
Family History of Mental Illness     object
dtype: object

## Data Types Conversion

In [12]:
data['Study Satisfaction'] = data['Study Satisfaction'].astype('category')
data['Job Satisfaction'] = data['Job Satisfaction'].astype('category')
data['Academic Pressure'] = data['Academic Pressure'].astype('category')
data['Work Pressure'] = data['Work Pressure'].astype('category')
data['Financial Stress'] = data['Financial Stress'].astype('category')
data['Work/Study Hours'] = data['Work/Study Hours'].astype('category')

## Checking for Missing Values

In [13]:
missing_values_cols = data.isnull()
print('Missing value counts:\n')
print(missing_values_cols.sum()[missing_values_cols.sum() > 0])
print('\nMissing value percentages:\n')
print(round((missing_values_cols.mean()[missing_values_cols.mean() > 0]) * 100, 2))

Missing value counts:

Profession            24632
Academic Pressure     75033
Work Pressure         18778
CGPA                  75034
Study Satisfaction    75033
Job Satisfaction      18774
Dietary Habits            5
Degree                    2
dtype: int64

Missing value percentages:

Profession            26.26
Academic Pressure     79.99
Work Pressure         20.02
CGPA                  79.99
Study Satisfaction    79.99
Job Satisfaction      20.01
Dietary Habits         0.01
Degree                 0.00
dtype: float64


### Basic Statistics
#### Continuous Variables

In [14]:
data.describe()

,Age,CGPA
count,93800.000000,18766.000000
mean,40.321685,7.674016
std,12.393480,1.465056
min,18.000000,5.030000
25%,29.000000,6.330000
50%,42.000000,7.800000
75%,51.000000,8.940000
max,60.000000,10.000000


**Insights:**
- The age distribution closely aligns with that observed in the training dataset, indicating consistency across samples
- Similar to the training dataset, CGPA data is available for only a subset of the population, likely reflecting the portion representing students
- The CGPA median is comparable to that in the training dataset, suggesting similar academic performance levels across the samples

In [15]:
categorical_variables = ['Gender', 'City', 'Working Professional or Student', 'Study Satisfaction',
                         'Job Satisfaction', 'Academic Pressure', 'Work Pressure', 'Work/Study Hours',
                         'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Financial Stress',
                         'Had Suicidal Thoughts?', 'Family History of Mental Illness']
data[categorical_variables].describe()

,Gender,City,Working Professional or Student,Study Satisfaction,Job Satisfaction,Academic Pressure,Work Pressure,Work/Study Hours,Profession,Sleep Duration,Dietary Habits,Degree,Financial Stress,Had Suicidal Thoughts?,Family History of Mental Illness
count,93800,93800,93800,18767.0,75026.0,18767.0,75022.0,93800.0,69168,93800,93795,93798,93800.0,93800,93800
unique,2,68,2,5.0,5.0,5.0,5.0,13.0,64,31,22,87,5.0,2,2
top,Male,Kalyan,Working Professional,4.0,2.0,3.0,2.0,10.0,Teacher,Less than 5 hours,Moderate,Class 12,2.0,No,No
freq,51262,4387,75028,4175.0,16971.0,4930.0,16255.0,9450.0,16385,25661,33018,9812,21151.0,47375,47118


**Insights:**

- **Gender:** The dataset has 93,800 entries for gender, with two categories (presumably Male and Female). The most frequent gender is Male, with 51,262 entries, indicating a gender imbalance similar to that in the training dataset.

- **City:** There are 68 unique cities, with "Kalyan" being the most common, appearing 4,387 times. This suggests a wide geographical representation, though certain cities, like Kalyan, have a higher concentration.

- **Working Professional or Student:** The dataset includes two categories, "Working Professional" and "Student." The majority are Working Professionals, with 75,028 entries, which aligns with the training dataset's focus on working individuals.

- **Study Satisfaction:** This attribute has five satisfaction levels, with Level 4 being the most common (4,171 entries). Data is available for a subset (18,767 entries), likely only for students, as observed in the training dataset.

- **Job Satisfaction:** Five job satisfaction levels are recorded, with Level 2 (16,971 entries) being the most common, suggesting moderate job dissatisfaction among working professionals, consistent with the training set. Data is available for 75,026 entries, aligning with the working professional group.

- **Academic Pressure:** The dataset includes five levels of academic pressure, with Level 3 (4,930 entries) being the most frequent, indicating moderate academic pressure for students. This attribute is limited to the student group (18,767 entries), as in the training dataset.

- **Work Pressure:** Five levels of work pressure are present, with Level 2 (16,255 entries) as the most frequent, suggesting moderate work pressure among working professionals, consistent with the training data. This attribute applies primarily to the working professional subset (75,022 entries).

- **Work/Study Hours:** There are 13 unique values for work/study hours, with 10 hours being the most common (9,450 entries), suggesting that long hours are prevalent in this sample population.

- **Profession:** The dataset includes 64 unique professions, with "Teacher" as the most common (16,385 entries), indicating a high representation of educators.

- **Sleep Duration:** The dataset records 31 unique sleep durations, with "Less than 5 hours" being the most frequent (25,661 entries), highlighting a trend of short sleep duration among participants.

- **Dietary Habits:** There are 22 unique dietary categories, with "Moderate" being the most common (33,018 entries), indicating that most individuals report average or balanced dietary habits.

- **Degree:** The dataset has 87 unique educational qualifications, with "Class 12" being the most common (9,812 entries), suggesting a significant portion of participants have completed secondary education.

- **Financial Stress:** Financial stress has five levels, with Level 2 being the most common (21,151 entries), indicating moderate financial stress, similar to the training dataset.

- **Had Suicidal Thoughts?:** This attribute has two responses, "Yes" and "No." "No" is the most frequent response (47,375 entries), suggesting that about half of the respondents reported no suicidal thoughts, similar to the training set.

- **Family History of Mental Illness:** Two responses, "Yes" and "No," are recorded for family history of mental illness. The most common response is "No" (47,118 entries), indicating that the majority do not have a known family history of mental illness.